In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from spacepy import pycdf
from datetime import datetime, timedelta
import os
from PIL import Image, ImageOps

In [108]:
fsmi_cdf = pycdf.CDF('data/cdfs/images/fsmi/thg_l1_asf_fsmi_2008020203_v01.cdf')
fsmi_cdf

<CDF:
range_epoch: CDF_EPOCH [2]
thg_asf_fsmi: CDF_UINT2 [1200, 256, 256]
thg_asf_fsmi_column: CDF_UINT2 [256] NRV
thg_asf_fsmi_epoch: CDF_EPOCH [1200]
thg_asf_fsmi_epoch0: CDF_EPOCH [] NRV
thg_asf_fsmi_row: CDF_UINT2 [256] NRV
thg_asf_fsmi_tend: CDF_REAL8 [1200]
thg_asf_fsmi_time: CDF_REAL8 [1200]
>

In [107]:
def preprocess_image(images):
    normalized_images = []
    for image in images:
        new_image = image - np.percentile(image, 1) # subtract 1st percentile
        new_image = new_image / np.percentile(image, 99) # devide by 99th percentile
        new_image = np.maximum(np.zeros(new_image.shape), new_image) # set negative to 0
        new_image = np.minimum(np.ones(new_image.shape), new_image) # set upper bound to 1
        new_image = (new_image * 255).astype(np.uint8)
        normalized_images.append(new_image)
    return normalized_images

In [105]:
from spacepy import pycdf
from datetime import datetime, timedelta
from PIL import Image, ImageOps

def convert_cdf_to_png(cdf, image_column, time_column, dest_dir):
    # load numpy array
    images = cdf[image_column]
    times = cdf[time_column]
    
    # preprocess images
    images = preprocess_image(images)
    
    # create destination directory
    os.mkdir(dest_dir)
    
    for image, time in zip(images, times):
        # file name in YYYYMMDDHHMMSS format
        time_str = time.strftime('%Y%m%d%H%M%S')
        
        # convert to PIL Image
        im = Image.fromarray(image)
        
        # grayscale
        im = ImageOps.grayscale(im)
        
        # save as png in destination format
        im.save(f'{dest_dir}/image_{time_str}.png')

In [110]:
dest_path = 'data/images/thg_asf_fsmi'

convert_cdf_to_png(fsmi_cdf, 'thg_asf_fsmi', 'thg_asf_fsmi_epoch', dest_path)